In [1]:
val spark = SparkSession.builder.appName("SampleKMeans") 
             .master("local[*]")
             .getOrCreate()
import spark.implicits._
import org.apache.spark.sql._
import org.apache.spark.ml.clustering._
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark._

spark = org.apache.spark.sql.SparkSession@68d24914


Spark local-1665504227098: Some(http://10.142.0.5:4049)

In [2]:
case class Person(name: String, age: Long)

// Encoders are created for case classes
import spark.implicits._
val caseClassDS = Seq(Person("Andy", 32)).toDS()
caseClassDS.show()

+----+---+
|name|age|
+----+---+
|Andy| 32|
+----+---+



defined class Person
caseClassDS = [name: string, age: bigint]


[name: string, age: bigint]

In [3]:
val path = "/data/spark/people.json"
val peopleDS = spark.read.json(path).as[Person]
peopleDS.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



path = /data/spark/people.json
peopleDS = [age: bigint, name: string]


[age: bigint, name: string]

In [4]:
val spark2 = spark
import spark2.implicits._

case class Person(name: String, age: Long)

val txtRDD = sc.textFile("/data/spark/people.txt")
val arrayRDD = txtRDD.map(_.split(","))
val personRDD = arrayRDD.map(attributes => Person(attributes(0), attributes(1).trim.toInt))
val peopleDF = personRDD.toDF()
peopleDF.show()

+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
+-------+---+



spark2 = org.apache.spark.sql.SparkSession@68d24914
defined class Person
txtRDD = /data/spark/people.txt MapPartitionsRDD[7] at textFile at <console>:24
arrayRDD = MapPartitionsRDD[8] at map at <console>:25
personRDD = MapPartitionsRDD[9] at map at <console>:26
peopleDF = [name: string, age: bigint]


[name: string, age: bigint]

In [5]:
import org.apache.spark.sql.types._
import org.apache.spark.sql._
// The schema is encoded in a string
val schemaString = "name age"
val fieldsArray = schemaString.split(" ")
val fields = fieldsArray.map(
    name => StructField(name, StringType, nullable = true)
    )
val schema = StructType(fields)
val peopleRDD = spark.sparkContext.textFile("/data/spark/people.txt")
val rowRDD = peopleRDD.map(_.split(",")).map(
    attributes => Row(attributes(0), attributes(1).trim)
    )
val peopleDF = spark.createDataFrame(rowRDD, schema)
peopleDF.show()

+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
+-------+---+



schemaString = name age
fieldsArray = Array(name, age)
fields = Array(StructField(name,StringType,true), StructField(age,StringType,true))
schema = StructType(StructField(name,StringType,true), StructField(age,StringType,true))
peopleRDD = /data/spark/people.txt MapPartitionsRDD[14] at textFile at <console>:65
rowRDD = MapPartitionsRDD[16] at map at <console>:66
peopleDF = [name: string, age: string]


[name: string, age: string]

In [6]:
%AddDeps com.databricks:spark-avro_2.11:4.0.0

Usage: %AddDeps my.company artifact-id version

Option                        Description                          
------                        -----------                          
--abort-on-resolution-errors  Abort (no downloads) when resolution 
                                fails                              
--classifier                  Sets the dependency's classifier     
--credential                  Adds a credential file to be used to 
                                the list                           
--exclude                     exclude dependency                   
--ivy-configuration           Sets the Ivy configuration for the   
                                dependency; defaults to "default"  
--repository                  Adds an additional repository to     
                                available list                     
--trace                       Prints out trace of download progress
--transitive                  Retrieve dependencies recursively    


In [11]:
//import spark2.implicits._
val df = spark.read.format("com.databricks.spark.avro").load("/data/spark/episodes.avro")
df.show()

Name: org.apache.spark.sql.AnalysisException
Message: Failed to find data source: com.databricks.spark.avro. Please find an Avro package at http://spark.apache.org/third-party-projects.html;
StackTrace:   at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:590)
  at org.apache.spark.sql.execution.datasources.DataSource.providingClass$lzycompute(DataSource.scala:86)
  at org.apache.spark.sql.execution.datasources.DataSource.providingClass(DataSource.scala:86)
  at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:325)
  at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:152)
  at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:135)

In [9]:
%AddDeps com.databricks spark-xml_2.11 0.12.0

Marking com.databricks:spark-xml_2.11:0.12.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir7592072976879876760/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir7592072976879876760/toree_add_deps/https/repo1.maven.org/maven2/com/databricks/spark-xml_2.11/0.12.0/spark-xml_2.11-0.12.0.jar
-> New file at /tmp/toree-tmp-dir7592072976879876760/toree_add_deps/https/repo1.maven.org/maven2/com/databricks/spark-xml_2.11/0.12.0/spark-xml_2.11-0.12.0.pom


lastException: Throwable = null


In [10]:
spark.read.format("xml").option("rowTag","book").load("/data/spark/books.xml").show()

+-----+--------------------+--------------------+---------------+-----+------------+--------------------+
|  _id|              author|         description|          genre|price|publish_date|               title|
+-----+--------------------+--------------------+---------------+-----+------------+--------------------+
|bk101|Gambardella, Matthew|


         An in...|       Computer|44.95|  2000-10-01|XML Developer's G...|
|bk102|          Ralls, Kim|A former architec...|        Fantasy| 5.95|  2000-12-16|       Midnight Rain|
|bk103|         Corets, Eva|After the collaps...|        Fantasy| 5.95|  2000-11-17|     Maeve Ascendant|
|bk104|         Corets, Eva|In post-apocalyps...|        Fantasy| 5.95|  2001-03-10|     Oberon's Legacy|
|bk105|         Corets, Eva|The two daughters...|        Fantasy| 5.95|  2001-09-10|  The Sundered Grail|
|bk106|    Randall, Cynthia|When Carla meets ...|        Romance| 4.95|  2000-09-02|         Lover Birds|
|bk107|      Thurman, Paula|A deep sea diver .